In [ ]:
# Cell 1: Setup (導入、配置、加載模型和 Tokenizer)
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import logging
import json
import os
import gc
import sys
import time # 用於模擬打字延遲
import re  # 引入正則表達式庫用於後處理

# --- 設定日誌 ---
logging.basicConfig(
    level=logging.INFO, # 默認 INFO，start_chat 中會設為 DEBUG
    format='[%(levelname)s] %(asctime)s %(name)s %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
# --- [修改] 更新 Logger 名稱 ---
logger = logging.getLogger("HUA-YI-INFERENCE-V6")

# --- 全局變量 ---
model = None
tokenizer = None
is_ready = False

# --- 配置 ---
base_model_name = "yentinglin/Llama-3-Taiwan-8B-Instruct"
# --- [關鍵修改] 指向新訓練模型的 Adapter 路徑 ---
adapter_path = "./huayi-yenting-llama3-lora-accelerate-v6-r32-e4-lr5e5" # <<<--- 確認這個路徑正確！
# --- [建議修改] 更新潛在的合併模型路徑名稱 ---
output_merged_model_path = "./huayi-yenting-llama3-merged-v6-r32-e4-lr5e5" # <<<--- 建議修改這裡！

load_in_4bit = True
merge_and_save = False # 在 Notebook 中，除非確定要合併，否則保持 False
use_merged_model_if_exists = True # 推薦 True

# --- 模型和 Tokenizer 加載函數 (與 v6 推理版本一致) ---
def load_model_and_tokenizer():
    """加載基礎模型、Adapter 和 Tokenizer (使用 v6 adapter 路徑)"""
    global model, tokenizer, is_ready
    if is_ready: logger.info("模型和 Tokenizer 已加載。"); return
    logger.info("開始加載模型和 Tokenizer...")
    merged_exists = os.path.isdir(output_merged_model_path)
    adapter_exists = os.path.isdir(adapter_path)
    if not adapter_exists and not merged_exists:
        logger.error(f"LoRA adapter 路徑 ({adapter_path}) 或合併模型路徑 ({output_merged_model_path}) 均未找到")
        raise FileNotFoundError(f"模型文件未找到，請檢查路徑: {adapter_path} 或 {output_merged_model_path}")
    quant_config = None
    model_load_kwargs = {"torch_dtype": torch.bfloat16, "device_map": "auto", "trust_remote_code": True}
    if load_in_4bit:
        quant_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_compute_dtype=torch.bfloat16, bnb_4bit_quant_type="nf4")
        model_load_kwargs["quantization_config"] = quant_config
        model_load_kwargs["low_cpu_mem_usage"] = True
        logger.info("使用 4-bit 量化加載模型")
    model_to_load = base_model_name
    load_adapter_later = False
    tokenizer_load_path = base_model_name
    if use_merged_model_if_exists and merged_exists:
        logger.info(f"從已合併模型 {output_merged_model_path} 加載...")
        model_to_load = output_merged_model_path
        tokenizer_load_path = output_merged_model_path
    elif adapter_exists:
        logger.info(f"從基礎模型 {base_model_name} 和 adapter {adapter_path} 加載...")
        load_adapter_later = True
    else: raise FileNotFoundError("無法確定有效的模型加載路徑")
    logger.info(f"正在加載模型: {model_to_load}...")
    try:
        import warnings
        with warnings.catch_warnings(): warnings.filterwarnings("ignore"); model = AutoModelForCausalLM.from_pretrained(model_to_load, **model_load_kwargs)
        logger.info("模型加載成功")
    except Exception as e: logger.error(f"加載模型失敗: {e}", exc_info=True); raise
    if load_adapter_later:
        logger.info(f"正在從 {adapter_path} 加載 LoRA adapter...")
        try:
            model = PeftModel.from_pretrained(model, adapter_path, is_trainable=False)
            logger.info(f"LoRA adapter 從 {adapter_path} 加載成功")
            if merge_and_save: # 合併邏輯 (merge_and_save=False 時不執行)
                logger.warning("Merge and Save 設為 True。"); logger.info("合併 LoRA 權重...")
                try:
                    model = model.merge_and_unload(); logger.info("權重合併完成")
                    if not os.path.exists(output_merged_model_path): os.makedirs(output_merged_model_path)
                    logger.info(f"保存合併模型到 {output_merged_model_path}...")
                    tokenizer_for_save = AutoTokenizer.from_pretrained(base_model_name, use_fast=True)
                    model.save_pretrained(output_merged_model_path); tokenizer_for_save.save_pretrained(output_merged_model_path)
                    logger.info("合併模型保存成功"); tokenizer_load_path = output_merged_model_path
                except Exception as e: logger.error(f"合併或保存失敗: {e}", exc_info=True); logger.warning("繼續使用未合併模型。"); tokenizer_load_path = base_model_name
        except Exception as e: logger.error(f"加載 LoRA adapter ({adapter_path}) 失敗: {e}", exc_info=True); raise
    logger.info(f"正在從 {tokenizer_load_path} 加載 Tokenizer...")
    try:
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_load_path, use_fast=True)
        if tokenizer.pad_token is None or tokenizer.pad_token_id is None or tokenizer.pad_token_id == tokenizer.eos_token_id:
            logger.warning(f"Tokenizer PAD token 未設置或等於 EOS。將使用 EOS token (ID: {tokenizer.eos_token_id}) 作為 PAD token。")
            tokenizer.pad_token = tokenizer.eos_token
            if tokenizer.pad_token_id is None: tokenizer.pad_token_id = tokenizer.eos_token_id
            if hasattr(model.config, 'pad_token_id') and model.config.pad_token_id != tokenizer.eos_token_id : model.config.pad_token_id = tokenizer.eos_token_id
        else:
             if tokenizer.pad_token is None and tokenizer.pad_token_id is not None:
                 try: tokenizer.pad_token = tokenizer.decode(tokenizer.pad_token_id)
                 except Exception as decode_err: logger.warning(f"無法解碼 pad_token_id {tokenizer.pad_token_id}: {decode_err}"); tokenizer.pad_token = f"<pad:{tokenizer.pad_token_id}>"
             logger.info(f"Tokenizer 使用預設 PAD token (ID: {tokenizer.pad_token_id})")
        tokenizer.padding_side = 'left'
        logger.info(f"Tokenizer 加載成功. Pad token: '{tokenizer.pad_token}', Pad token ID: {tokenizer.pad_token_id}, Padding side: {tokenizer.padding_side}")
    except Exception as e: logger.error(f"加載 Tokenizer 失敗: {e}", exc_info=True); raise
    model.eval(); is_ready = True
    logger.info(f"模型 (來自 {model_to_load} + adapter {adapter_path if load_adapter_later else 'N/A'}) 和 Tokenizer 已成功加載並準備就緒！")

# --- 推理函數 (Prompt v5.1 - 修正 f-string 錯誤) ---
def create_inference_prompt(persona, goal, conversation_history, last_message):
    """根據輸入創建推理用的 Prompt, 加入對話歷史, 極端強調風格 (v5.6 - 修正 f-string 錯誤)"""
    prompt_parts = []
    # --- 使用與訓練 v6 一致的風格指令 ---
    style_instruction = """### **[絕對規則] 目標輸出風格 (MUST FOLLOW RULES):**
*   **核心要求 (Core Requirement)**: 100% 模仿台灣大學生用 LINE 聊天。輸出**必須極度簡短**！每個概念都必須拆成獨立的短訊息，並且**只能**使用換行符 `\\n` 來分隔這些短訊息。
*   **絕對禁止 (ABSOLUTELY FORBIDDEN - DO NOT USE):**
    *   **任何 Emoji**。 *   **逗號** `,`。 *   **句號** `.` 或 `。`。 *   **引號** `"` `'` 「」 『』。
    *   **頓號** `、`。 *   **分號** `;` 或 `；`。 *   **冒號** `:` 或 `：`。 *   **括號** `()` `（）` `[]`。
    *   **任何項目符號或列表標記**。 *   **任何 "話翼:" 或類似的角色/機器人名稱前綴**。
*   **唯一允許的標點 (ONLY ALLOWED PUNCTUATION):**
    *   **空格** ` `。 *   **換行符** `\\n`。 *   **問號** `？` (少量)。 *   **驚嘆號** `！` (少量)。
*   **訊息長度 (Message Length)**: 總回覆非常簡短。1 到 4 個 `\\n` (2 到 5 則短訊息)。
*   **輸出格式範例 (Output Format Examples):**
    *   範例 1:\n喔喔\n原來是這樣
    *   範例 2:\n欸\n那你想去哪\n我看看時間
    *   範例 3:\n哈哈\n真的假的\n笑死"""
    prompt_parts.append(style_instruction)
    if persona: prompt_parts.append(f"### 對方資訊 (Persona):\n{persona}")
    if goal: prompt_parts.append(f"### 你的對話目標 (Your Goal):\n{goal}")
    context_str = ""
    if conversation_history and isinstance(conversation_history, list):
        history_limit = 6; limited_history = conversation_history[-history_limit:]
        formatted_history = []
        for entry in limited_history:
            if entry.startswith("對方："): formatted_history.append(entry)
            elif entry.startswith("話翼："):
                simplified_reply = entry[3:].strip().replace('\n', ' ')
                formatted_history.append(f"你：{simplified_reply}")
            else: formatted_history.append(entry)
        context_str += "\n".join(formatted_history)
    if last_message:
        last_msg_clean = str(last_message).replace('\n', ' ')
        if context_str: context_str += f"\n對方剛說：{last_msg_clean}"
        else: context_str = f"對方剛說：{last_msg_clean}" # 只有 last_message 時

    # --- [修正 v5.6] ---
    # 處理只有 last_message 沒有 dialogue_context 的情況
    if context_str: # 如果 context_str 不為空 (即有 dialogue_context 或 last_message 已被加入)
        prompt_parts.append(f"### 對話紀錄 (Context):\n{context_str}")
    elif last_message: # 如果 context_str 為空但有 last_message
        last_msg_clean_single = str(last_message).replace('\n', ' ') # 確保單行
        prompt_parts.append(f"### 對方剛說的話:\n{last_msg_clean_single}") # 使用修正後的 f-string
    # --- 修正結束 ---

    prompt_content = "\n\n".join(prompt_parts)
    formatted_prompt = (
        f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n"
        f"參考下方所有資訊，並**嚴格遵守**上面列出的**所有絕對規則**，生成一個建議回覆。\n\n"
        f"{prompt_content}\n\n"
        f"**直接生成**完全符合**所有**風格規則的建議回覆："
        f"<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    )
    return formatted_prompt

# --- 推理函數 (v5.5 base - 參數可調) ---
def generate_reply(prompt,
                   max_new_tokens=60,
                   temperature=0.3,
                   top_p=0.7,
                   do_sample=True,
                   repetition_penalty=1.15,
                   no_repeat_ngram_size=3):
    """使用模型生成回覆建議 (v5.5 base - 參數可調)"""
    global model, tokenizer
    if not is_ready: return "[模型未加載]"
    if not prompt: return "[Prompt 為空]"
    try:
        if not isinstance(prompt, str): logger.error(f"Prompt type error: {type(prompt)}"); return "[Prompt 類型錯誤]"
        inputs = tokenizer(prompt, return_tensors="pt", padding=False, add_special_tokens=False).to(model.device)
        if inputs.input_ids.shape[1] == 0: logger.warning("Tokenization failed"); return "[Tokenization 失敗]"
        if 'attention_mask' not in inputs: inputs['attention_mask'] = torch.ones_like(inputs['input_ids'])
        eos_token_ids_set = {tokenizer.eos_token_id}
        try:
            eot_id = tokenizer.convert_tokens_to_ids("<|eot_id|>")
            if eot_id is not None and eot_id != tokenizer.unk_token_id: eos_token_ids_set.add(eot_id); logger.debug(f"Added <|eot_id|> ({eot_id})")
            elif eot_id == tokenizer.unk_token_id: logger.warning("'<|eot_id|>' resolves to UNK")
        except KeyError: logger.warning("Tokenizer doesn't know '<|eot_id|>'")
        except Exception as e: logger.error(f"Error processing '<|eot_id|>' token: {e}", exc_info=True)
        final_eos_ids = list(eos_token_ids_set)
        logger.debug(f"Using EOS token IDs: {final_eos_ids}")
        generate_kwargs = {
            "input_ids": inputs.input_ids, "attention_mask": inputs.attention_mask,
            "max_new_tokens": max_new_tokens, "temperature": temperature, "top_p": top_p,
            "do_sample": do_sample, "pad_token_id": tokenizer.pad_token_id,
            "eos_token_id": final_eos_ids, "repetition_penalty": repetition_penalty,
            "no_repeat_ngram_size": no_repeat_ngram_size,
        }
        logger.info(f"開始生成回覆 (temp={temperature}, top_p={top_p}, max_new={max_new_tokens})...")
        with torch.no_grad():
            with torch.amp.autocast(device_type=model.device.type, dtype=torch.bfloat16): outputs = model.generate(**generate_kwargs)
        input_length = inputs.input_ids.shape[1]
        if outputs.shape[1] <= input_length: logger.warning("模型未生成任何新的 token。"); reply = ""
        else: generated_ids = outputs[0][input_length:]; reply = tokenizer.decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        logger.info("生成完成")
        return reply.strip()
    except Exception as e: logger.error(f"生成回覆時出錯: {e}", exc_info=True); import traceback; logger.error(traceback.format_exc()); return "[生成錯誤]"

logger.info("推理輔助函數已定義 (v5.5 base)。")

# --- 對話狀態管理 ---
conversation_history = []
# --- 使用與訓練 v6 一致的溫和 Goal ---
current_persona = "25歲正在就讀科技法律研究所的女生，大學是吉他社，很喜歡日本文化，安全型人格，住在台北，討厭把家庭順位放太高的男生，討厭父權男，不喜歡在小事情上計較"
current_goal = "目前我跟這個女生是朋友，我想要跟他變成更好的朋友" # <<<--- 保持溫和目標

# --- 交互式對話循環 (v5.4 base - 使用 v5.5 generate_reply, v5.3 後處理) ---
def start_chat():
    """啟動交互式對話循環 (使用 v6 模型)"""
    global conversation_history
    logger.setLevel(logging.DEBUG)
    logger.warning("Logger level set to DEBUG for this chat session.")
    # --- [修改] 更新版本號 ---
    print("\n=========================================")
    print("  話翼 AI 社交教練 - Jupyter 對話模式 v6 (fix 5.6)") # 更新版本號
    print(f"    (Adapter: {adapter_path})")
    print("=========================================")
    print("\n指令: quit, reset, history")
    print("-" * 40)
    print(f"[Persona: {current_persona}]")
    print(f"[Goal: {current_goal}]")
    print("-" * 40)

    # --- 後處理正則 (與 v5.3/v5.4 一致，移除了 Emoji 過濾) ---
    prefix_pattern = re.compile(r'^\s*(?:話翼|AI|Assistant|模型|回答|建議)\s*[:：]?\s*', re.IGNORECASE)
    forbidden_punctuation_list = ",.，。、；：「」『』（）《》\"'();:" # 保持移除這些

    while True:
        try:
            print("\n" + "=" * 20)
            print(f"[歷史記錄長度: {len(conversation_history)}]")
            try: user_input = input(">>> 你 (對方剛說的話 或 指令): ").strip()
            except (EOFError, KeyboardInterrupt): print("\n檢測到退出信號..."); user_input = "quit"
            if user_input.lower() == 'quit': print("\n正在退出..."); break
            elif user_input.lower() == 'reset':
                conversation_history = []; logger.info("對話已重置")
                print("\n[對話已重置]\n"); print("-" * 40); print(f"[Persona: {current_persona}]"); print(f"[Goal: {current_goal}]"); print("-" * 40); continue
            elif user_input.lower() == 'history':
                 print("\n--- 對話歷史 ---");
                 if conversation_history: [print(f"{i+1}: {msg}") for i, msg in enumerate(conversation_history)]
                 else: print("(空)")
                 print("----------------"); continue
            if not user_input: print("  [提示] 請輸入對方說的話..."); continue

            # 使用 v5.6 Prompt (修正了 f-string 錯誤)
            prompt = create_inference_prompt(persona=current_persona, goal=current_goal, conversation_history=conversation_history, last_message=user_input)
            logger.debug(f"Generated Prompt (first 500 chars):\n---\n{prompt[:500]}...\n---")

            # 使用 v5.5 generate_reply
            raw_ai_reply = generate_reply(prompt=prompt)
            logger.debug(f"Raw AI Reply: '{raw_ai_reply}'")

            # --- 後處理步驟 (v5.4 邏輯) ---
            cleaned_reply_step1 = prefix_pattern.sub('', raw_ai_reply).strip(); logger.debug(f"After Step 1 (Prefix): '{cleaned_reply_step1}'")
            logger.debug("Skipping Step 2 (Emoji)")
            cleaned_reply_step3 = cleaned_reply_step1; removed_punctuations = []
            for punc in forbidden_punctuation_list:
                if punc in cleaned_reply_step3: removed_punctuations.append(punc); cleaned_reply_step3 = cleaned_reply_step3.replace(punc, '')
            if removed_punctuations: logger.debug(f"Removed punctuations: {' '.join(removed_punctuations)}")
            logger.debug(f"After Step 3 (Punctuation): '{cleaned_reply_step3}'")

            cleaned_reply_step4 = cleaned_reply_step3.replace('\\n', '\n')
            logger.debug(f"After Step 4 (Newline Replace): '{cleaned_reply_step4}'")

            processed_reply = cleaned_reply_step4

            lines = processed_reply.split('\n')
            cleaned_lines = []
            logger.debug(f"Processing lines (count={len(lines)}):")
            for i, line in enumerate(lines):
                stripped_line = line.strip()
                logger.debug(f"  Line {i}: Raw='{line}' -> Stripped='{stripped_line}'")
                if stripped_line: cleaned_lines.append(stripped_line)
                else: logger.debug(f"  Line {i} was empty after stripping, discarded.")

            final_reply = "\n".join(cleaned_lines)
            logger.debug(f"Final Reply after joining (num lines={len(cleaned_lines)}): '{final_reply}'")

            # --- 打印處理後的回覆 ---
            print(f"\n✨ 話翼建議:")
            if not final_reply:
                print("(模型未生成有效回覆 或 回覆被過濾)")
                logger.warning(f"最終回覆為空。原始回覆: '{raw_ai_reply}'")
                final_reply_for_history = "[過濾後為空]"
            else:
                lines_to_print = final_reply.split('\n')
                for i, line in enumerate(lines_to_print): print(line); time.sleep(0.2) if i < len(lines_to_print) - 1 else None
                final_reply_for_history = final_reply

            # 更新對話歷史
            conversation_history.append(f"對方：{user_input}"); conversation_history.append(f"話翼：{final_reply_for_history}")
            history_limit = 8
            if len(conversation_history) > history_limit: conversation_history = conversation_history[-history_limit:]

        except Exception as e: logger.error(f"對話循環中發生錯誤: {e}", exc_info=True); print("\n發生內部錯誤，請檢查日誌。")

    logger.setLevel(logging.INFO); logger.info("Chat session ended. Logger level reset to INFO.")
    print("\n對話結束！")

# --- 主執行區塊 ---
if __name__ == "__main__":
    print("此腳本設計為在 Jupyter Notebook 中運行。")
    print("請按順序運行 Cell 來加載模型和啟動對話。")
    print("1. 運行包含此代碼的 Cell (定義函數)。")
    print("2. 在新 Cell 中運行: load_model_and_tokenizer()")
    print("3. 在新 Cell 中運行: start_chat()")

# --- 使用說明 ---
# ... (同上)

[INFO] 2025-05-06 16:41:33 HUA-YI-INFERENCE-V6 推理輔助函數已定義 (v5.5 base)。


此腳本設計為在 Jupyter Notebook 中運行。
請按順序運行 Cell 來加載模型和啟動對話。
1. 運行包含此代碼的 Cell (定義函數)。
2. 在新 Cell 中運行: load_model_and_tokenizer()
3. 在新 Cell 中運行: start_chat()


In [ ]:
load_model_and_tokenizer()
start_chat()

[INFO] 2025-05-06 16:41:33 HUA-YI-INFERENCE-V6 開始加載模型和 Tokenizer...
[INFO] 2025-05-06 16:41:33 HUA-YI-INFERENCE-V6 使用 4-bit 量化加載模型
[INFO] 2025-05-06 16:41:33 HUA-YI-INFERENCE-V6 從基礎模型 yentinglin/Llama-3-Taiwan-8B-Instruct 和 adapter ./huayi-yenting-llama3-lora-accelerate-v6-r32-e4-lr5e5 加載...
[INFO] 2025-05-06 16:41:33 HUA-YI-INFERENCE-V6 正在加載模型: yentinglin/Llama-3-Taiwan-8B-Instruct...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

[INFO] 2025-05-06 16:44:04 HUA-YI-INFERENCE-V6 模型加載成功
[INFO] 2025-05-06 16:44:04 HUA-YI-INFERENCE-V6 正在從 ./huayi-yenting-llama3-lora-accelerate-v6-r32-e4-lr5e5 加載 LoRA adapter...
/trinity/home/tna001/.local/lib/python3.9/site-packages/peft/peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight', 'base_m


  話翼 AI 社交教練 - Jupyter 對話模式 v6 (fix 5.6)
    (Adapter: ./huayi-yenting-llama3-lora-accelerate-v6-r32-e4-lr5e5)

指令: quit, reset, history
----------------------------------------
[Persona: 25歲正在就讀科技法律研究所的女生，大學是吉他社，很喜歡日本文化，安全型人格，住在台北，討厭把家庭順位放太高的男生，討厭父權男，不喜歡在小事情上計較]
[Goal: 目前我跟這個女生是朋友，我想要跟他變成更好的朋友]
----------------------------------------

[歷史記錄長度: 0]


>>> 你 (對方剛說的話 或 指令):  早安


...

[DEBUG] 2025-05-06 16:45:29 HUA-YI-INFERENCE-V6 Generated Prompt (first 500 chars):
---
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

參考下方所有資訊，並**嚴格遵守**上面列出的**所有絕對規則**，生成一個建議回覆。

### **[絕對規則] 目標輸出風格 (MUST FOLLOW RULES):**
*   **核心要求 (Core Requirement)**: 100% 模仿台灣大學生用 LINE 聊天。輸出**必須極度簡短**！每個概念都必須拆成獨立的短訊息，並且**只能**使用換行符 `\n` 來分隔這些短訊息。
*   **絕對禁止 (ABSOLUTELY FORBIDDEN - DO NOT USE):**
    *   **任何 Emoji**。 *   **逗號** `,`。 *   **句號** `.` 或 `。`。 *   **引號** `"` `'` 「」 『』。
    *   **頓號** `、`。 *   **分號** `;` 或 `；`。 *   **冒號** `:` 或 `：`。 *   **括號** `()` `（）` `[]`。
    *   **任何項...
---
[DEBUG] 2025-05-06 16:45:29 HUA-YI-INFERENCE-V6 Added <|eot_id|> (128009)
[DEBUG] 2025-05-06 16:45:29 HUA-YI-INFERENCE-V6 Using EOS token IDs: [128009]
[INFO] 2025-05-06 16:45:29 HUA-YI-INFERENCE-V6 開始生成回覆 (temp=0.3, top_p=0.7, max_new=60)...
[INFO] 2025-05-06 16:45:31 HUA-YI-INFERENCE-V6 生成完成
[DEBUG] 2025-05-06 16:45:31 HUA-YI-INFERENCE-V6 Raw AI Reply: '欸 老師 真酷耶'
[DEBUG] 2025-05-06 16:45:31 HUA-YI-I


✨ 話翼建議:
欸 老師 真酷耶

[歷史記錄長度: 6]


>>> 你 (對方剛說的話 或 指令):  那你呢


[DEBUG] 2025-05-06 16:45:54 HUA-YI-INFERENCE-V6 Generated Prompt (first 500 chars):
---
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

參考下方所有資訊，並**嚴格遵守**上面列出的**所有絕對規則**，生成一個建議回覆。

### **[絕對規則] 目標輸出風格 (MUST FOLLOW RULES):**
*   **核心要求 (Core Requirement)**: 100% 模仿台灣大學生用 LINE 聊天。輸出**必須極度簡短**！每個概念都必須拆成獨立的短訊息，並且**只能**使用換行符 `\n` 來分隔這些短訊息。
*   **絕對禁止 (ABSOLUTELY FORBIDDEN - DO NOT USE):**
    *   **任何 Emoji**。 *   **逗號** `,`。 *   **句號** `.` 或 `。`。 *   **引號** `"` `'` 「」 『』。
    *   **頓號** `、`。 *   **分號** `;` 或 `；`。 *   **冒號** `:` 或 `：`。 *   **括號** `()` `（）` `[]`。
    *   **任何項...
---
[DEBUG] 2025-05-06 16:45:54 HUA-YI-INFERENCE-V6 Added <|eot_id|> (128009)
[DEBUG] 2025-05-06 16:45:54 HUA-YI-INFERENCE-V6 Using EOS token IDs: [128009]
[INFO] 2025-05-06 16:45:54 HUA-YI-INFERENCE-V6 開始生成回覆 (temp=0.3, top_p=0.7, max_new=60)...
[INFO] 2025-05-06 16:45:58 HUA-YI-INFERENCE-V6 生成完成
[DEBUG] 2025-05-06 16:45:58 HUA-YI-INFERENCE-V6 Raw AI Reply: '欸 那你現在在幹嘛
我在準備期中考試
還沒畢業耶'
[DEBUG] 2025-05-06 1


✨ 話翼建議:
欸 那你現在在幹嘛
我在準備期中考試
還沒畢業耶

[歷史記錄長度: 8]
